In [ ]:
# | default_exp ds/config
# | export
from pydantic import StringConstraints, model_validator, BaseModel
from pydantic.dataclasses import dataclass, Field

from datetime import datetime, date

from typing import Annotated

from space_analysis.plot.config import FigureExtraOptions

In [ ]:
# | export
@dataclass
class DataConfig:
    """Data configuration"""

    timerange: list[date | datetime] = None

    ds: str = None
    id: str = None
    name: str = None
    units: str = None
    labels: list[str] = Field(default_factory=list)

    satellite: Annotated[str, StringConstraints(to_lower=True)] = None
    instrument: Annotated[str, StringConstraints(to_lower=True)] = None
    datatype: str = None
    probe: str = None


class DataConfigSatellite(BaseModel):
    """Data configuration for one satellite"""

    timerange: list[date | datetime] = None
    ids: list[str] = Field(default_factory=list)

    satellite: str = None
    instruments: list[str] = Field(default_factory=list)
    probe: str = None

    @property
    def data_configs(self):
        if len(self.instruments) == len(self.ids):
            return [
                DataConfig(
                    **dict(self.model_dump(exclude_none=True), id=id, instrument=inst)
                )
                for id, inst in zip(self.ids, self.instruments)
            ]

In [ ]:
# | export
@dataclass
class OutputConfig:
    """Output configuration"""

    path: str = None
    filename: str = None

    formats: list[str] = Field(default_factory=list)
    display: bool = False

    figure: dict = Field(default_factory=dict)
    """Figure options"""

    figure_extra: FigureExtraOptions = Field(default_factory=FigureExtraOptions)
    """Extra figure options"""


@dataclass
class PanelConfig:
    """Panel configuration"""

    timerange: list[date | datetime] = None

    ds: str = None
    id: str = None
    name: str = None
    units: str = None
    labels: list[str] = Field(default_factory=list)

    satellite: Annotated[str, StringConstraints(to_lower=True)] = None
    instrument: Annotated[str, StringConstraints(to_lower=True)] = None
    datatype: str = None
    probe: str = None

In [ ]:
# | export
@dataclass
class Config:
    panels: list[PanelConfig]
    timerange: list[datetime | date] = None
    output: OutputConfig = None
    backend: str = None

    @model_validator(mode="before")
    def set_default_timerange(cls, values):
        timerange = values.get("timerange", None)
        panels = values.get("panels", [])

        if timerange:
            for panel in panels:
                panel["timerange"] = panel.get("timerange") or timerange
        return values